In [1]:
import os

# 환경 변수 설정
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import random
import pandas as pd
import numpy as np


import gymnasium as gym
from gymnasium import spaces
import pybamm
from gymnasium import Env


from stable_baselines3 import DDPG, PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.noise import NormalActionNoise

In [26]:
class DFN(Env):
    def __init__(self, render_mode=None):
        option = {"thermal":"lumped"}
        self.model = pybamm.lithium_ion.SPMe(option)
        self.SoC_desired = 0.8
        self.r_max_temp = 273 + 30
        self.r_max_volt = 4.2
        self.Time_goal = 60
        self.SoC = 0.2
        self.ep_num = 0
        # Define observation space and action space
        self.observation_space = spaces.Box(low=0, high=400, shape=(3,), dtype=np.float32)
        self.action_space = spaces.Box(dtype=np.float32, low=0, high=10, shape=(1,))

        # Initialize PyBaMM model and simulation
        self.parameter_values = pybamm.ParameterValues("Chen2020")
        update_input = ({
            'Number of cells connected in series to make a battery': 4.0,
            'Number of electrodes connected in parallel to make a cell': 4.0,
            'Upper voltage cut-off [V]': 5,})


        self.parameter_values.update(update_input)
        self.solution = []
        self.temp = 0
        self.volt = 0
        
    def _get_obs(self):
        #return np.array([self.SoC])
        return np.array([self.SoC, self.volt, self.temp], dtype=np.float32)  
    
    def _get_info(self):
        return {"distance": self.SoC_desired - self.SoC}    
    def get_parameter_values(self):
        parms = self.parameter_values.copy()
        return parms
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        option = {"thermal":"lumped"}
        model = pybamm.lithium_ion.SPMe(option)
        parms = self.parameter_values.copy()
        self.solution = []
        # Reset SoC and start a new simulation
        self.SoC = 0.2
        experiment = pybamm.Experiment(["Rest for 30 min"])
        sim = pybamm.Simulation(model, experiment=experiment,parameter_values = parms)
        step_solution = sim.solve(initial_soc = 0.2)

        self.solution += [step_solution.last_state]
        observation = self._get_obs()
        info = self._get_info()
        self.ep_num +=1
        print(self.ep_num)
        return observation, info

    def step(self, action):
        option = {"thermal":"lumped"}
        model = pybamm.lithium_ion.SPMe(option)
        parms = self.parameter_values.copy()
        if self.SoC >= self.SoC_desired:
            terminated = True

        else:
             terminated = False
        charge_rate = float(action)
        # Perform one step in the simulation with the given action (charge rate)
        #try:

            # experiment = pybamm.Experiment([f"Charge at {charge_rate}A for 30 sec or until 4.2 V"])
        experiment = pybamm.Experiment([f"Charge at {charge_rate}C for 30 sec or until 4.2 V"])
        sim = pybamm.Simulation(model, experiment=experiment, parameter_values = parms)
        step_solution = sim.solve(starting_solution=self.solution[-1].last_state)


        self.solution +=[step_solution]
        

        # Calculate reward based on various factors
        self.SoC -= step_solution["Discharge capacity [A.h]"].entries[-1] / self.parameter_values["Nominal cell capacity [A.h]"]
        self.volt = step_solution["Terminal voltage [V]"].entries[-1]
        self.temp = step_solution["X-averaged cell temperature [K]"].entries[-1]
        r_temp = -5 * abs(self.temp - self.r_max_temp) if self.temp > self.r_max_temp else 0
        r_volt = -100 * abs(self.volt - self.r_max_volt) if self.volt > self.r_max_volt else 0
        r_fast = -0.1
        reward = r_fast + r_temp + r_volt

        # except:
        #     terminated = True
        #     reward = -1000
        observation = self._get_obs()
        info = self._get_info()
        print(self.volt,self.temp,self.SoC,reward)
        #print(observation,reward, terminated, False, info)
        # print(len(self.solution))
        # Check if termination condition is met
        

        return observation, reward, terminated, False, info

    def render(self, mode='human'):
        pass


In [31]:
env = DFN()

In [32]:
env.reset()

1


(array([0.2, 0. , 0. ], dtype=float32), {'distance': 0.6000000000000001})

In [33]:
env.step(0.1)

SolverError: Step 'Charge at 0.1C for 30 sec or until 4.2 V' is infeasible due to exceeded bounds at initial conditions. If this step is part of a longer cycle, round brackets should be used to indicate this, e.g.:
 pybamm.Experiment([(
	Discharge at C/5 for 10 hours or until 3.3 V,
	Charge at 1 A until 4.1 V,
	Hold at 4.1 V until 10 mA
])

In [30]:
a = env.get_parameter_values()
a

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x00000155FFC2C3A0>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x00000155FFC2C310>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initi

In [9]:
env = DFN()
# It will check your custom environment and output additional warnings if needed
check_env(env)  

1
2


SolverError: Step 'Charge at 3.4108448028564453C for 30 sec or until 4.2 V' is infeasible due to exceeded bounds at initial conditions. If this step is part of a longer cycle, round brackets should be used to indicate this, e.g.:
 pybamm.Experiment([(
	Discharge at C/5 for 10 hours or until 3.3 V,
	Charge at 1 A until 4.1 V,
	Hold at 4.1 V until 10 mA
])

In [4]:
# env = DFN()
# model = PPO('MlpPolicy', env)
# model.learn(total_timesteps=(20))
# model.save("batt")

In [5]:
# env = DFN()
# model = PPO('MlpPolicy', env)
# model.learn(total_timesteps=(6000000))
# model.save("batt")

In [6]:
# env = DFN()
# model = PPO.load("batt",env=env)

# vec_env = model.get_env()
# obs = vec_env.reset()

In [7]:
# b = True
# while b:
#     action, _states = model.predict(obs)
#     observation, reward, terminated, a, info = env.step(action)
#     print(observation, reward, terminated, a, info)
#     b = terminated